# Entrez Utilities (eutils)


BiomedQuery.Entrez provides an interface to some of the functionality in the [Entrez Utility API](https://www.ncbi.nlm.nih.gov/books/NBK25501/). 

The following E-utils functions have been implemented:

- ESearch
- EFetch
- ELink
- ESummary

The following utility functions are available to handle and store NCBI responses

- EParse - Convert XML response to Julia Dict
- Saving NCBI Responses to XML
- Saving EFetch to a SQLite database
- Saving EFetch to a MySQL database

The following utility functions are available to query the database

- All PMIDs
- All MESH descriptors for an article


## Import the Module and Environment Variables


In [1]:
using BioMedQuery.Entrez
email = ENV["NCBI_EMAIL"];
umls_user = ENV["UMLS_USER"];
umls_psswd = ENV["UMLS_PSSWD"];

## 1. esearch

esearch(search_dict): Requests a list of UIDS matchin a query. The input is a dictionary specifying all requiered parameters specified in the Entrez documentation [NCBI Entrez:Esearch](http://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.ESearch).

For instance, let's request 10 pmids for papers matching the query: (asthma[MeSH Terms]) AND ("2001/01/29"[Date - Publication] : "2010"[Date - Publication])

In [2]:
search_term = """(asthma[MeSH Terms]) AND ("2001/01/29"[Date - Publication] : "2010"[Date - Publication])"""
search_dic = Dict("db"=>"pubmed", "term" => search_term,
"retstart" => 0, "retmax"=>10,
"email" => email)
esearch_response = esearch(search_dic)

NCBI Response: OK


"<?xml version=\"1.0\" encoding=\"UTF-8\" ?>\n<!DOCTYPE eSearchResult PUBLIC \"-//NLM//DTD esearch 20060628//EN\" \"https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20060628/esearch.dtd\">\n<eSearchResult><Count>33664</Count><RetMax>10</RetMax><RetStart>0</RetStart><IdList>\n<Id>22525997</Id>\n<Id>22166637</Id>\n<Id>22166627</Id>\n<Id>22153142</Id>\n<Id>22055375</Id>\n<Id>22018425</Id>\n<Id>22017947</Id>\n<Id>21944487</Id>\n<Id>21928556</Id>\n<Id>21854724</Id>\n</IdList><TranslationSet><Translation>     <From>asthma[MeSH Terms]</From>     <To>\"asthma\"[MeSH Terms]</To>    </Translation></TranslationSet><TranslationStack>   <TermSet>    <Term>\"asthma\"[MeSH Terms]</Term>    <Field>MeSH Terms</Field>    <Count>115064</Count>    <Explode>Y</Explode>   </TermSet>   <TermSet>    <Term>\"2001/01/29\"[PDAT]</Term>    <Field>PDAT</Field>    <Count>0</Count>    <Explode>N</Explode>   </TermSet>   <TermSet>    <Term>\"2010\"[PDAT]</Term>    <Field>PDAT</Field>    <Count>0</Count>    <Explode>N</Expl

### Save the response to file

In [3]:
using XMLconvert
xmlASCII2file(esearch_response, "./esearch.xml");

### Convert to a Julia (Multi) Dictionary

In [4]:
esearch_dict = eparse(esearch_response)
println("Type of esearch_dict: ", typeof(esearch_dict))
show_key_structure(esearch_dict)

Type of esearch_dict: DataStructures.MultiDict{Any,Any}
    -TranslationStack
        -TermSet
            -Term
            -Field
            -Count
            -Explode
        -TermSet
            -Term
            -Field
            -Count
            -Explode
        -TermSet
            -Term
            -Field
            -Count
            -Explode
        -OP
        -OP
        -OP
    -QueryTranslation
    -RetStart
    -TranslationSet
        -Translation
            -From
            -To
    -IdList
        -Id
        -Id
        -Id
        -Id
        -Id
        -Id
        -Id
        -Id
        -Id
        -Id
    -Count
    -RetMax


### Flatten into Dictionary for easy access

In [5]:
flat_easearch_dict = flatten(esearch_dict)
display(flat_easearch_dict)

Dict{Any,Any} with 12 entries:
  "QueryTranslation"                 => Any["\"asthma\"[MeSH Terms] AND (\"2001…
  "RetStart"                         => Any[0]
  "TranslationStack-TermSet-Count"   => Any[115064,0,0]
  "TranslationStack-TermSet-Field"   => Any["MeSH Terms","PDAT","PDAT"]
  "IdList-Id"                        => Any[22525997,22166637,22166627,22153142…
  "TranslationStack-TermSet-Explode" => Any["Y","N","N"]
  "Count"                            => Any[33664]
  "RetMax"                           => Any[10]
  "TranslationSet-Translation-From"  => Any["asthma[MeSH Terms]"]
  "TranslationStack-TermSet-Term"    => Any["\"asthma\"[MeSH Terms]","\"2001/01…
  "TranslationStack-OP"              => Any["RANGE","GROUP","AND"]
  "TranslationSet-Translation-To"    => Any["\"asthma\"[MeSH Terms]"]

### Get all pmids returned by esearch

In [6]:
ids = Array{Int64,1}(flat_easearch_dict["IdList-Id" ])

10-element Array{Int64,1}:
 22525997
 22166637
 22166627
 22153142
 22055375
 22018425
 22017947
 21944487
 21928556
 21854724

## 2. efetch

In [7]:
# define the fetch dictionary
fetch_dic = Dict("db"=>"pubmed","tool" =>"BioJulia",
"email" => "maria_restrepo@brown.edu", "retmode" => "xml", "rettype"=>"null")

# fetch
efetch_response = efetch(fetch_dic, ids)

NCBI Response: OK


"<?xml version=\"1.0\" ?>\n<!DOCTYPE PubmedArticleSet PUBLIC \"-//NLM//DTD PubMedArticle, 1st January 2017//EN\" \"https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_170101.dtd\">\n<PubmedArticleSet>\n<PubmedArticle>\n    <MedlineCitation Status=\"MEDLINE\" Owner=\"NLM\">\n        <PMID Version=\"1\">22525997</PMID>\n        <DateCreated>\n            <Year>2012</Year>\n            <Month>04</Month>\n            <Day>24</Day>\n        </DateCreated>\n        <DateCompleted>\n            <Year>2012</Year>\n            <Month>09</Month>\n            <Day>10</Day>\n        </DateCompleted>\n        <DateRevised>\n            <Year>2015</Year>\n            <Month>11</Month>\n            <Day>19</Day>\n        </DateRevised>\n        <Article PubModel=\"Print-Electronic\">\n            <Journal>\n                <ISSN IssnType=\"Electronic\">1532-656X</ISSN>\n                <JournalIssue CitedMedium=\"Internet\">\n                    <Volume>26</Volume>\n                    <Issue>3</Issue>\n 

### Convert to XML respose to (Multi) Dictionary

In [8]:
efetch_dict = eparse(efetch_response)
show_key_structure(efetch_dict)

    -PubmedArticle
        -PubmedData
            -ArticleIdList
                -ArticleId
                    -ArticleId
                    -IdType
                -ArticleId
                    -ArticleId
                    -IdType
                -ArticleId
                    -ArticleId
                    -IdType
            -PublicationStatus
            -History
                -PubMedPubDate
                    -Day
                    -Month
                    -PubStatus
                    -Year
                -PubMedPubDate
                    -Day
                    -Month
                    -PubStatus
                    -Year
                -PubMedPubDate
                    -Day
                    -Month
                    -PubStatus
                    -Year
                -PubMedPubDate
                    -Day
                    -Hour
                    -Month
                    -PubStatus
                    -Minute
                    -Year
          

## 3. Save to MySQL

In [9]:
db_config = Dict(:host=>"127.0.0.1",
                 :dbname=>"biomed_query_test",
                 :username=>"root",
                 :pswd=>"bcbi123",
                 :overwrite=>true)

db = save_efetch_mysql(efetch_dict, db_config)

Initializing MySQL Database
/home/bcbi/.julia/v0.5/BioMedQuery/src/Entrez/create_pubmed_db.sql


 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in readall(::IOStream) at ./deprecated.jl:30
 in init_pubmed_db_mysql(::Dict{Symbol,Any}) at /home/bcbi/.julia/v0.5/BioMedQuery/src/Entrez/entrez_db.jl:37
 in save_efetch_mysql(::DataStructures.MultiDict{Any,Any}, ::Dict{Symbol,Any}, ::Bool) at /home/bcbi/.julia/v0.5/BioMedQuery/src/Entrez/entrez_save.jl:64
 in save_efetch_mysql(::DataStructures.MultiDict{Any,Any}, ::Dict{Symbol,Any}) at /home/bcbi/.julia/v0.5/BioMedQuery/src/Entrez/entrez_save.jl:63
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/bcbi/.julia/v0.5/IJulia/src/execute_request.jl:157
 in eventloop(::ZMQ.Socket) at /home/bcbi/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##13#19)() at ./task.jl:360
while loading In[9], in expression starting on line 7


Set to overwrite MySQL database biomed_query_test
Secure file *******************
1×2 DataFrames.DataFrame
│ Row │ Variable_name      │ Value                   │
├─────┼────────────────────┼─────────────────────────┤
│ 1   │ "secure_file_priv" │ "/var/lib/mysql-files/" │
Secure file *******************
Database biomed_query_test created and initialized
Saving 10 articles to database


MySQL Handle
------------
Host: 127.0.0.1
User: root
DB:   biomed_query_test


### Explore the MySQL Results Database

In [10]:
using MySQL
tables = mysql_execute(db, "show tables;")
display(tables)
articles = mysql_execute(db, "select * from article limit 10")
display(articles)
authors = mysql_execute(db, "select * from author limit 10")
display(authors)

,Tables_in_biomed_query_test
1,article
2,author
3,author2article
4,mesh_descriptor
5,mesh_heading
6,mesh_qualifier


,pmid,title,pubYear
1,21854724,[Asthma prevalence in adolescents. Relation to sex and active smoking].,2010
2,21928556,Exercise intolerance in obese children--is it asthma?,2010
3,21944487,[Skin prick tests with standardized extracts of mites of different precedence in patients with asthma and allergic rhinitis].,2010
4,22017947,[Occupational asthma in the Tunisian central region: etiologies and professional status].,2010
5,22018425,Coping patterns in Latino families of children with asthma.,2010
6,22055375,Making education count: the nurse's role in asthma education using a medical home model of care.,2010
7,22153142,Shared medical appointments: facilitating care for children with asthma and their caregivers.,2010
8,22166627,Prevalence of sensitivity to cockroach allergens and IgE cross-reactivity between cockroach and house dust mite allergens in Chinese patients with allergic rhinitis and asthma.,2010
9,22166637,"1,25-dihydroxyvitamin D₃ pretreatment enhances the efficacy of allergen immunotherapy in a mouse allergic asthma model.",2010
10,22525997,Are urban low-income children from unplanned pregnancy exposed to higher levels of environmental tobacco smoke?,2010


,id,forename,lastname
1,24,A,Abbassi
2,25,A J,Mlaouah
3,19,Adrienne,Garro
4,35,Alexis,Labrada Rosado
5,8,Bao-Qing,Sun
6,10,Birgitte,Gjesing
7,3,Bonita,Stanton
8,7,Chang-Zheng,Wang
9,13,Constance L,Wall-Haas
10,18,Dawn,Bolyard


## 4. Save as publications

In [11]:
citation_config = Dict(:type => "bibtex", :output_file => "citations_test.bib", :overwrite=>true)
    save_article_citations(efetch_dict, citation_config);

Saving citation for 10 articles


# BioMedQuery.Processes

The library comes with a series a "pre-assembled" workflows. For instance, we often need to call esearc, efetch and save to database as a pipeline.

In [12]:
using BioMedQuery.Processes

### esearch, efetch, mysql_save in one line of code

In [15]:
db = pubmed_search_and_save(email, search_term, 10,
    save_efetch_mysql, db_config);

Getting 10 articles, starting at index 0
------Searching Entrez--------
NCBI Response: OK
------Fetching Entrez--------
NCBI Response: OK
------Saving to database--------
Initializing MySQL Database
/home/bcbi/.julia/v0.5/BioMedQuery/src/Entrez/create_pubmed_db.sql
Set to overwrite MySQL database biomed_query_test
Secure file *******************
1×2 DataFrames.DataFrame
│ Row │ Variable_name      │ Value                   │
├─────┼────────────────────┼─────────────────────────┤
│ 1   │ "secure_file_priv" │ "/var/lib/mysql-files/" │
Secure file *******************
Database biomed_query_test created and initialized
Saving 10 articles to database
Finished searching, total number of articles: 10


### esearch, efetch, save citations in one line of code

In [21]:
pubmed_search_and_save(email, search_term, 10,
    save_article_citations, citation_config);

Getting 10 articles, starting at index 0
------Searching Entrez--------
NCBI Response: OK
------Fetching Entrez--------
NCBI Response: OK
------Saving to database--------
Saving citation for 10 articles
Finished searching, total number of articles: 10


Int64